# Section 1.3: Circuits - Moments and Circuit Construction

This notebook explores how to assemble quantum algorithms using Moments and Circuits in Cirq. You'll learn different circuit construction strategies and understand how to optimize circuit depth for NISQ hardware.

## Learning Objectives

- Understand Moments as parallel quantum operations
- Master circuit construction techniques
- Use Insert Strategies to optimize circuit depth
- Manipulate and analyze circuits
- Build common quantum circuit patterns

In [1]:
import cirq
import numpy as np
import matplotlib.pyplot as plt

## 1. Moments - Parallel Quantum Operations

A **Moment** represents operations executed during the same time slice. All operations in a moment must act on disjoint qubits, modeling quantum hardware parallelism.

In [6]:
q0, q1, q2 = cirq.LineQubit.range(3)

# Valid moment - disjoint qubits
moment1 = cirq.Moment([cirq.H(q0), cirq.X(q1), cirq.Y(q2)])
print(f"Moment:")
print(moment1)
print(f"Number of operations: {len(moment1)}")
print(f"Operations execute simultaneously (in parallel)")

Moment:
  ╷ 0 1 2
╶─┼───────
0 │ H X Y
  │
Number of operations: 3
Operations execute simultaneously (in parallel)


In [8]:
# Moment with two-qubit gate
moment2 = cirq.Moment([cirq.CNOT(q0, q1), cirq.H(q2)])
print(f"Moment:")
print(moment2)
print(f"CNOT uses q0 and q1, H uses q2 - all disjoint")

Moment:
  ╷ 0 1 2
╶─┼───────
0 │ @─X H
  │
CNOT uses q0 and q1, H uses q2 - all disjoint


In [9]:
# Iterating over moment
print("Operations in moment:")
for i, op in enumerate(moment1):
    print(f"  Operation {i}: {op}")

Operations in moment:
  Operation 0: H(q(0))
  Operation 1: X(q(1))
  Operation 2: Y(q(2))


In [10]:
# Invalid moment example
print("Attempting to create invalid moment (same qubit twice):")
try:
    invalid_moment = cirq.Moment([cirq.H(q0), cirq.X(q0)])
except ValueError as e:
    print(f"  Error: {e}")

Attempting to create invalid moment (same qubit twice):
  Error: Overlapping operations: (cirq.H(cirq.LineQubit(0)), cirq.X(cirq.LineQubit(0)))


## 2. Circuit Construction

A **Circuit** is an ordered sequence of Moments. Multiple ways to construct circuits:

In [11]:
# Empty circuit
empty_circuit = cirq.Circuit()
print(f"Empty circuit: {empty_circuit}")
print(f"Length (moments): {len(empty_circuit)}")

Empty circuit: 
Length (moments): 0


In [12]:
# Circuit from operations
q0, q1 = cirq.LineQubit.range(2)
circuit_ops = cirq.Circuit(cirq.H(q0), cirq.CNOT(q0, q1))
print("Circuit from operations:")
print(circuit_ops)

Circuit from operations:
0: ───H───@───
          │
1: ───────X───


In [13]:
# Circuit from moments
moment1 = cirq.Moment([cirq.H(q0)])
moment2 = cirq.Moment([cirq.CNOT(q0, q1)])
circuit_moments = cirq.Circuit(moment1, moment2)
print("Circuit from moments:")
print(circuit_moments)
print("The famous Bell State. If you measure one qbit, you automatically know the state of the other")

Circuit from moments:
0: ───H───@───
          │
1: ───────X───


In [18]:
# Building circuit by appending operations
circuit_append = cirq.Circuit()
circuit_append.append(cirq.H(q0))
circuit_append.append(cirq.H(q1))
circuit_append.append(cirq.CNOT(q0, q1))
print("Circuit built by appending:")
print(circuit_append)
print("The 'plus plus state', |+>|+>. In this specific case, the CNOT gate in the second moment has no effect. The resulting state is a special state (an eigenvector) that is immune to a CNOT gate. So the final state of the circuit is still just |+>|+>.")

Circuit built by appending:
0: ───H───@───
          │
1: ───H───X───
The 'plus plus state', |+>|+>. In this specific case, the CNOT gate in the second moment has no effect. The resulting state is a special state (an eigenvector) that is immune to a CNOT gate. So the final state of the circuit is still just |+>|+>.


## 3. Insert Strategies

Insert strategies determine how operations are placed into moments. This is critical for optimizing circuit depth on NISQ hardware.

### 3.1 NEW_THEN_INLINE Strategy (Default)

Adds to most recent moment if qubits are available, otherwise creates a new moment.

In [19]:
q0, q1, q2 = cirq.LineQubit.range(3)

circuit_new = cirq.Circuit()
circuit_new.append(cirq.H(q0), strategy=cirq.InsertStrategy.NEW_THEN_INLINE)
circuit_new.append(cirq.H(q1), strategy=cirq.InsertStrategy.NEW_THEN_INLINE)
circuit_new.append(cirq.X(q0), strategy=cirq.InsertStrategy.NEW_THEN_INLINE)
circuit_new.append(cirq.H(q2), strategy=cirq.InsertStrategy.NEW_THEN_INLINE)

print("Circuit with NEW_THEN_INLINE:")
print(circuit_new)
print(f"Depth (moments): {len(circuit_new)}")

Circuit with NEW_THEN_INLINE:
0: ───H───────X───────

1: ───────H───────────

2: ───────────────H───
Depth (moments): 4


### 3.2 EARLIEST Strategy

Searches backward to find earliest moment where operation fits. Creates more compact circuits.

In [20]:
circuit_earliest = cirq.Circuit()
circuit_earliest.append(cirq.H(q0), strategy=cirq.InsertStrategy.EARLIEST)
circuit_earliest.append(cirq.H(q1), strategy=cirq.InsertStrategy.EARLIEST)
circuit_earliest.append(cirq.X(q0), strategy=cirq.InsertStrategy.EARLIEST)
circuit_earliest.append(cirq.H(q2), strategy=cirq.InsertStrategy.EARLIEST)

print("Circuit with EARLIEST:")
print(circuit_earliest)
print(f"Depth (moments): {len(circuit_earliest)}")

Circuit with EARLIEST:
0: ───H───X───

1: ───H───────

2: ───H───────
Depth (moments): 2


In [21]:
print(f"Comparison:")
print(f"  NEW_THEN_INLINE depth: {len(circuit_new)}")
print(f"  EARLIEST depth: {len(circuit_earliest)}")
print(f"  EARLIEST is more compact, reducing circuit execution time")

Comparison:
  NEW_THEN_INLINE depth: 4
  EARLIEST depth: 2
  EARLIEST is more compact, reducing circuit execution time


## 4. Circuit Manipulation and Inspection

Cirq provides rich APIs for analyzing and manipulating circuits.

In [22]:
# Create a sample circuit
q0, q1, q2 = cirq.LineQubit.range(3)
circuit = cirq.Circuit()
circuit.append([cirq.H(q0), cirq.H(q1)])
circuit.append(cirq.CNOT(q0, q1))
circuit.append(cirq.X(q2))
circuit.append(cirq.CNOT(q1, q2))

print("Sample circuit:")
print(circuit)

Sample circuit:
0: ───H───@───────
          │
1: ───H───X───@───
              │
2: ───X───────X───


In [23]:
# Circuit depth
print(f"Circuit depth (number of moments): {len(circuit)}")

# All operations
print(f"\nAll operations:")
for i, op in enumerate(circuit.all_operations()):
    print(f"  {i}: {op}")

# All qubits
print(f"\nAll qubits used: {sorted(circuit.all_qubits())}")

Circuit depth (number of moments): 3

All operations:
  0: H(q(0))
  1: H(q(1))
  2: X(q(2))
  3: CNOT(q(0), q(1))
  4: CNOT(q(1), q(2))

All qubits used: [cirq.LineQubit(0), cirq.LineQubit(1), cirq.LineQubit(2)]


In [24]:
# Iterate over moments
print("Moments in circuit:")
for i, moment in enumerate(circuit):
    print(f"  Moment {i}: {list(moment)}")

Moments in circuit:
  Moment 0: [cirq.H(cirq.LineQubit(0)), cirq.H(cirq.LineQubit(1)), cirq.X(cirq.LineQubit(2))]
  Moment 1: [cirq.CNOT(cirq.LineQubit(0), cirq.LineQubit(1))]
  Moment 2: [cirq.CNOT(cirq.LineQubit(1), cirq.LineQubit(2))]


In [25]:
# Circuit slicing
sliced = circuit[0:2]
print("Circuit slicing (first 2 moments):")
print(sliced)

Circuit slicing (first 2 moments):
0: ───H───@───
          │
1: ───H───X───

2: ───X───────


In [26]:
# Circuit concatenation
circuit_extra = cirq.Circuit(cirq.measure(q0, q1, q2, key='result'))
combined = circuit + circuit_extra
print("Combined circuit:")
print(combined)

Combined circuit:
0: ───H───@───────M('result')───
          │       │
1: ───H───X───@───M─────────────
              │   │
2: ───X───────X───M─────────────


## 5. Common Circuit Patterns

Let's explore typical quantum circuit patterns.

### 5.1 Bell State Preparation

In [27]:
q0, q1 = cirq.LineQubit.range(2)
bell_circuit = cirq.Circuit(
    cirq.H(q0),
    cirq.CNOT(q0, q1)
)
print("Bell State Circuit:")
print(bell_circuit)
print("\nPrepares: (|00⟩ + |11⟩)/√2")

Bell State Circuit:
0: ───H───@───
          │
1: ───────X───

Prepares: (|00⟩ + |11⟩)/√2


### 5.2 GHZ State Preparation

In [28]:
q0, q1, q2, q3 = cirq.LineQubit.range(4)
ghz_circuit = cirq.Circuit()
ghz_circuit.append(cirq.H(q0))
ghz_circuit.append(cirq.CNOT(q0, q1))
ghz_circuit.append(cirq.CNOT(q1, q2))
ghz_circuit.append(cirq.CNOT(q2, q3))

print("GHZ State Circuit (4 qubits):")
print(ghz_circuit)
print("\nPrepares: (|0000⟩ + |1111⟩)/√2")

GHZ State Circuit (4 qubits):
0: ───H───@───────────
          │
1: ───────X───@───────
              │
2: ───────────X───@───
                  │
3: ───────────────X───

Prepares: (|0000⟩ + |1111⟩)/√2


### 5.3 Parallel Operations

In [29]:
qubits = cirq.LineQubit.range(4)
parallel_circuit = cirq.Circuit()
parallel_circuit.append([cirq.H(q) for q in qubits])

print("Parallel Hadamards:")
print(parallel_circuit)
print(f"\nDepth: {len(parallel_circuit)} (single moment)")
print(f"Operations: {len(list(parallel_circuit.all_operations()))}")

Parallel Hadamards:
0: ───H───

1: ───H───

2: ───H───

3: ───H───

Depth: 1 (single moment)
Operations: 4


The GHZ circuit is a "chain" that creates entanglement step-by-step, resulting in a deep circuit (4 Moments). The Parallel example is a "wide" circuit that applies the same gate to all qubits at once, resulting in a shallow circuit (1 Moment).

## 6. Circuit Statistics

Analyze circuit properties for optimization.

In [30]:
# Create sample circuit
qubits = cirq.LineQubit.range(4)
circuit = cirq.Circuit()
circuit.append(cirq.H(qubits[0]))
circuit.append([cirq.H(q) for q in qubits[1:]])
circuit.append(cirq.CNOT(qubits[0], qubits[1]))
circuit.append(cirq.CNOT(qubits[2], qubits[3]))
circuit.append(cirq.X(qubits[0]))

print("Sample circuit:")
print(circuit)

# Count operations
all_ops = list(circuit.all_operations())
print(f"\nTotal operations: {len(all_ops)}")

# Count two-qubit gates
two_qubit_gates = sum(1 for op in all_ops if len(op.qubits) == 2)
print(f"Two-qubit gates: {two_qubit_gates}")

# Count single-qubit gates
single_qubit_gates = sum(1 for op in all_ops if len(op.qubits) == 1)
print(f"Single-qubit gates: {single_qubit_gates}")

# Qubits used
print(f"Qubits used: {sorted(circuit.all_qubits())}")
print(f"Number of unique qubits: {len(circuit.all_qubits())}")

# Depth
print(f"Circuit depth (moments): {len(circuit)}")

Sample circuit:
0: ───H───@───X───
          │
1: ───H───X───────

2: ───H───@───────
          │
3: ───H───X───────

Total operations: 7
Two-qubit gates: 2
Single-qubit gates: 5
Qubits used: [cirq.LineQubit(0), cirq.LineQubit(1), cirq.LineQubit(2), cirq.LineQubit(3)]
Number of unique qubits: 4
Circuit depth (moments): 3


In [31]:
# Operations per moment
print("\nOperations per moment:")
for i, moment in enumerate(circuit):
    print(f"  Moment {i}: {len(moment)} operations - {list(moment)}")


Operations per moment:
  Moment 0: 4 operations - [cirq.H(cirq.LineQubit(0)), cirq.H(cirq.LineQubit(1)), cirq.H(cirq.LineQubit(2)), cirq.H(cirq.LineQubit(3))]
  Moment 1: 2 operations - [cirq.CNOT(cirq.LineQubit(0), cirq.LineQubit(1)), cirq.CNOT(cirq.LineQubit(2), cirq.LineQubit(3))]
  Moment 2: 1 operations - [cirq.X(cirq.LineQubit(0))]


## Exercises

Try these modifications:

1. **Optimize a circuit**: Create a circuit with many sequential single-qubit gates and use EARLIEST strategy to reduce depth.

2. **Build W state**: Create a circuit that prepares the 3-qubit W state: (|100⟩ + |010⟩ + |001⟩)/√3

3. **Compare strategies**: Build the same algorithm with both NEW_THEN_INLINE and EARLIEST. Compare depths.

4. **Circuit analysis**: Given a circuit, count how many entangling gates it contains.

## Key Takeaways

- **Moments** represent parallel operations on disjoint qubits
- **Circuits** are ordered sequences of moments
- **Insert strategies** (EARLIEST vs NEW_THEN_INLINE) critically impact circuit depth
- Circuit depth directly affects execution time and decoherence on NISQ hardware
- Cirq provides rich APIs for circuit inspection and manipulation
- Common patterns (Bell, GHZ) are building blocks for algorithms

Next: [Section 1.4 - Execution](part1_section_1_4_execution.ipynb)